# VANGUARD AB TEST


## METADATA HELP

This comprehensive set of fields will guide your analysis, helping you unravel the intricacies of client behavior and preferences.

- **client_id**: Every client’s unique ID.
- **variation**: Indicates if a client was part of the experiment.
- **visitor_id**: A unique ID for each client-device combination.
- **visit_id**: A unique ID for each web visit/session.
- **process_step**: Marks each step in the digital process.
- **date_time**: Timestamp of each web activity.
- **clnt_tenure_yr**: Represents how long the client has been with Vanguard, measured in years.
- **clnt_tenure_mnth**: Further breaks down the client’s tenure with Vanguard in months.
- **clnt_age**: Indicates the age of the client.
- **gendr**: Specifies the client’s gender.
- **num_accts**: Denotes the number of accounts the client holds with Vanguard.
- **bal**: Gives the total balance spread across all accounts for a particular client.
- **calls_6_mnth**: Records the number of times the client reached out over a call in the past six months.
- **logons_6_mnth**: Reflects the frequency with which the client logged onto Vanguard’s platform over the last six months.


In [49]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from cleaning import *
from mining import *
from db_handling import *
import pandas as pd
from dotenv import load_dotenv
import os


### Load Configuration

In [51]:
# Load config.yaml
config = parse_config()

{'database_name': 'vanguard_ab', 'refresh_db': False, 'tables': {'clients': {'sources': [{'path': 'data/df_final_demo.txt', 'url': 'https://github.com/data-bootcamp-v4/lessons/blob/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt'}], 'separator': ',', 'columns': {'client_id': {'original_name': 'client_id', 'data_type': 'INTEGER', 'primary_key': True, 'pandas_dtype': 'int64'}, 'client_since_year': {'original_name': 'clnt_tenure_yr', 'data_type': 'FLOAT', 'pandas_dtype': 'int64'}, 'client_since_month': {'original_name': 'clnt_tenure_mnth', 'data_type': 'FLOAT', 'pandas_dtype': 'int64'}, 'client_age': {'original_name': 'clnt_age', 'data_type': 'FLOAT', 'pandas_dtype': 'int64'}, 'gender': {'original_name': 'gendr', 'data_type': 'CHAR(1)', 'pandas_dtype': 'category', 'valid_categories': ['U', 'M', 'F'], 'fallback_category': 'U'}, 'number_of_accounts': {'original_name': 'num_accts', 'data_type': 'FLOAT', 'pandas_dtype': 'int64'}, 'balance': {'original_name': 'bal', 

## Data Mining

In [52]:
#TODO: adapt function for remote urls
#TODO: local caching
#TODO: looks for source files, if not found, fetch from source, clean, and save

In [53]:
# Creates a dictionary of all imported dataframes
dataframes = { name:import_data_from_config(config, name) for name in config['tables']}

## Data Cleaning

In [54]:
#TODO: don't impose categories?

In [55]:
# Rename columns
dataframes = rename_columns(dataframes, config)

In [56]:
# Select columns
dataframes = select_columns(dataframes, config)

In [57]:
# Data Categorizing
dataframes = clean_categorical_data(dataframes, config)

In [58]:
#Convert types
dataframes = convert_types(dataframes, config)

In [59]:
display_dataFrames(dataframes,'head','dtypes', 'cat_count')

clients:
clients - Head:


,client_id,client_since_year,client_since_month,client_age,gender,number_of_accounts,balance,calls_6_months,logons_6_month
0,836976,6,73,60,U,2,45105.30,6,9
1,2304905,7,94,58,U,2,110860.30,6,9
2,1439522,5,64,32,U,2,52467.79,6,9
3,1562045,16,198,49,M,2,67454.65,3,6
4,5126305,12,145,33,F,2,103671.75,0,3


clients - Dtypes:


client_id                int64
client_since_year        int64
client_since_month       int64
client_age               int64
gender                category
number_of_accounts       int64
balance                float64
calls_6_months           int64
logons_6_month           int64
dtype: object

clients - Cat_count:


...

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


visits - Dtypes:


client_id                int64
visitor_id              object
visit_id                object
process_step          category
date_time       datetime64[ns]
dtype: object

visits - Cat_count:


,process_step
process_step,
start,217820
step_1,146864
step_2,123536
step_3,107256
confirm,90806
unknown,0


In [60]:
# Handle duplicates

In [61]:
# Handle missing values

### SQL EXPORT

In [62]:
# Load environment variables
load_dotenv()
db_password = os.getenv('SQL_PASSWORD')

In [63]:
# Create database if it doesn't exist
engine = create_db(db_password, config)

In [64]:
# Export tables to database if refresh is set to true
export_dataframes_to_sql(engine, dataframes, config)

Skipping export, set refresh_db to true in config.yaml to export to SQL.


## Data Re-import

In [65]:
# Import data from database
cleaned_dfs = import_all_tables_from_sql(engine)

### Local Caching

In [66]:
# Save files locally in an untracked folder
export_dataframes_to_csv(cleaned_dfs)

In [67]:
clients_df = import_data(['data/cleaned/clients.csv'])
experiment_df = import_data(['data/cleaned/experiment.csv'])
visits_df = import_data(['data/cleaned/visits.csv'])

AttributeError: 'str' object has no attribute 'get'

In [ ]:
display('clients :',clients_df, 'experiment :',experiment_df, 'visits :',visits_df)

## Data Exploration

In [22]:
# Handle outliers

In [23]:
#frequency tables

## Analysis

In [24]:
#TODO: consider binning / pd.cut / qcut for numerical data
#TODO: correlation matrix
#TODO: tukeys_test_outliers

In [ ]:
# check back and forth between steps, lost?
# 

## Visualizations

## Conclusions